In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np

In [3]:
import nltk
# nltk.download('stopwords') #run once 
from nltk.corpus import stopwords 

In [8]:
query = "adani"

tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i>20000:
        break
    else:
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])

df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('sentiment.csv', mode = 'a')

<generator object TwitterSearchScraper.get_items at 0x7f1391507060>


In [4]:
eng_stop_words = list(stopwords.words('english'))

In [ ]:
# regex